In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt 
from scipy.signal import savgol_filter
from astropy.stats import sigma_clipped_stats as scs
import numpy as np
import glob


In [2]:
lc = fits.open("../xsm/data/2021/11/11/calibrated/ch2_xsm_20211111_v1_level2.lc")

FileNotFoundError: [Errno 2] No such file or directory: '../xsm/data/2021/11/11/calibrated/ch2_xsm_20211111_v1_level2.lc'

In [ ]:
lc.info()

## Search for outliers

### Background subtraction 

In [ ]:
counts = lc[1].data['RATE']
time = lc[1].data['TIME']

In [ ]:
plt.plot(time, counts)

Need to split the .lc file into contiguous orbits

In [ ]:
def orbit_start_indices(time, step=1):
    output_arr = []
    length_arr = []
    for i in range(len(time)):
        if(i==0):
            output_arr.append(i)
        elif(time[i]-time[i-1]>step):
            length_arr.append(time[i-1]-time[output_arr[-1]])
            output_arr.append(i)
    return output_arr, length_arr

In [ ]:
orbit_start_indices(time, 2)

In [ ]:
def distinct_orbits(arr, indices_array):
    output_arr = []
    for i in range(len(indices_array)):
        if(i==0):
            temp = arr[indices_array[0]:indices_array[1]]
            output_arr.append(temp)
        elif(i<len(indices_array)-1):
            temp = arr[indices_array[i]:indices_array[i+1]]
            output_arr.append(temp)
        else:
            temp = arr[indices_array[-1]:]
            output_arr.append(temp)
    return output_arr

In [ ]:
def n_sigma(time, counts, n):
    '''n-sigma : returns indices where counts>(mean+n*sigma)
    Returns flags, mean, sigma'''
    mean,_, sigma = scs(counts)
    #flags = np.where(counts>(mean+n*sigma)
    flags = np.where(counts>(mean+n*sigma))
    return flags, mean, sigma

Need to rebin the data according to the minimum duration of burst being observed

In [ ]:
def bin_edges_from_time(time, t_bin):
    time = np.array(time)
    bin_edges = (time[1:] + time[:-1])/2.0
    bin_edges = np.insert(bin_edges, 0, bin_edges[0] - t_bin)
    bin_edges = np.append(bin_edges,bin_edges[-1] + t_bin)
    return bin_edges

In [ ]:
'''
flags, mean, sigma = n_sigma(time, counts, 3)
for orbit_num in range(len(time_split)):
    #run search on this
    plt.figure(orbit_num)
    print(mean, sigma)
    plt.plot(time_split[orbit_num], counts_split[orbit_num])
    plt.axhline(mean+3*sigma,linestyle='--', color='r')
    plt.axhline(mean, linestyle = '--', color='g')
'''

Splitting the lightcurve into orbits will not work as it becomes difficult to distinguish background variations from the actual solar burst (ask about duration, min, max)

In [ ]:
plt.figure(0)
flags, mean, sigma = n_sigma(time, counts, 3)
plt.plot(time, counts, alpha=0.7 )
plt.axhline(mean+3*sigma,linestyle='--', color='r')
plt.axhline(mean, linestyle = '--', color='g')
plt.scatter(time[flags], counts[flags])

Linear background fit 

In [ ]:
from scipy.stats import linregress

In [ ]:
slope, intercept, r, p, se = linregress(time, counts)

In [ ]:
def background_corrected(time, counts, mode='linear'):
    '''time, rates 
    Returns background corrected rates'''
    if(mode == 'linear'):
        #linear for now 
        slope, intercept, r, p, se = linregress(time, counts)
        return counts - (slope*time + intercept)
    elif(mode == 'constant'):
        mean, _ , _ = scs(counts)
        return counts - mean

In [ ]:
plt.figure(1)
plt.plot(time, background_corrected(time, counts))

Below function is piecewise 

In [ ]:
def piece(bin_edges, bin_rates, t_bin):
    time = np.arange(bin_edges[0], bin_edges[-1], t_bin)
    #binning
    bin_widths = (bin_edges[1:] - bin_edges[:1])
    condlist=np.zeros((bin_rates.size, time.size), dtype=float)
    for i in range(bin_rates.size):
    #making condlist
        condlist[i] = (time>=bin_edges[i])&(time<bin_edges[i+1])
    rates = np.piecewise(x=time, condlist=condlist, funclist=bin_rates)
    return time, rates

## Make a new binning function 

In [ ]:
def rebin_lc(time, rates, t_bin, t_bin_new):
    #t_bin_new is the new binning
    """time, rates
    t_bin : original binning in time
    t_bin_new : desired binning
    Returns time, rates (counts/s)"""
    new_time = np.arange(time[0]-t_bin/2 + t_bin_new/2, time[-1]+t_bin/2 + t_bin_new/2, t_bin_new)
    bin_edges = bin_edges_from_time(new_time, t_bin_new)
    bin_counts = np.histogram(time, bins = bin_edges, weights = rates)[0]
    bin_widths = bin_edges[1:] - bin_edges[:-1]
    bin_rates = bin_counts/bin_widths
    return new_time, bin_rates

In [ ]:
f_time = np.arange(0,20,1)
f_rates = np.random.randint(0, 5, size=(20))
new_time = np.arange(f_time[0]-1.0/2 + 4.0/2, f_time[-1]+1.0/2 - 4.0/2 + 4.0, 4.0)
bin_edges = bin_edges_from_time(new_time, 4.0)
#bin_edges = (new_time[1:]+new_time[:-1])/2
bin_counts = np.histogram(f_time, bins = bin_edges, weights = f_rates)[0]
bin_widths = bin_edges[1:] - bin_edges[:-1]
bin_rates = bin_counts/bin_widths

In [ ]:
new_time

In [ ]:
bin_edges

In [ ]:
plt.plot(f_time, f_rates, '-o', drawstyle = 'steps-mid')
plt.plot(new_time, bin_rates, '-o',drawstyle = 'steps-mid')
plt.xlim(0,20)
plt.grid()

In [ ]:
counts = background_corrected(time , counts)

In [ ]:
time_new, counts_new = rebin_lc(time, counts, 1.0, 25.0)

In [ ]:
plt.figure(0, figsize=(15,5))
n=3
flags, mean, sigma = n_sigma(time_new, counts_new, n)
plt.scatter(time_new, counts_new, alpha=0.7)
plt.plot(time, counts, alpha=0.2)
plt.axhline(mean+n*sigma,linestyle='--', color='r')
plt.axhline(mean, linestyle = '--', color='g')
plt.scatter(time_new[flags], counts_new[flags])
#plt.xlim(1.533e8+74000, 1.533e8+77000)

In [ ]:
time_new[flags]

In [ ]:
orb_idx, len_arr = orbit_start_indices(np.array(time_new[flags]).flatten(), 25.0)

In [ ]:
orb_idx

In [ ]:
len_arr

In [ ]:
orb_idx

In [ ]:
burst_start_idx = int(np.argmax(len_arr))
flags=np.array(flags).flatten()
burst_flags = flags[orb_idx[burst_start_idx] : orb_idx[burst_start_idx+1]]

In [ ]:
plt.scatter(time_new[burst_flags], counts_new[burst_flags])

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
from scipy.special import erf

In [ ]:
def EFP(x, A, B, C, D):
    Z = (2*B + C**2*D)/(2*C)
    return 1/2 * np.sqrt(np.pi) *  A * C * np.exp(D*(B-x) + C**2*D**2/4) * (erf(Z) - erf(Z - x/C))

In [ ]:
flags2 = np.arange(581, 628)

def EFP(x, A, B, C, D):
    Z = (2*B + C**2*D)/(2*C)
    return 1/2 * np.sqrt(np.pi) *  A * C * np.exp(D*(B-x) + C**2*D**2/4) * (erf(Z) - erf(Z - x/C))

popt, pcov = curve_fit(EFP, np.float128(time_new[flags2]), np.float128(counts_new[flags2]), p0 =([25, 6000+1.5337e8, 17, 0.1]))

#x2 = np.arange(1.533e8+75000, 1.533e8+77000)
x2 = np.arange(1.533e8+74000, 1.533e8+78000)
plt.scatter(time_new[flags2], counts_new[flags2])
plt.plot(x2, EFP(x2, *popt))
plt.savefig("efp_fit_1.png")

Should we ensure mean zero while fitting?

In [ ]:
plt.plot(x2, EFP(x2, *popt))
plt.axhline(0+sigma,linestyle='--', color='r', label = "1$\sigma$")
plt.axhline(0, linestyle = '--', color='g')
#plt.scatter(time_new[flags2], counts_new[flags2])
plt.legend()

In [ ]:
t_arr = x2[np.argwhere(np.diff(np.sign(EFP(x2, *popt) - sigma))).flatten()]
t_start = t_arr[0]
t_end = t_arr[-1]
t_max = x2[np.argmax(EFP(x2, *popt))]

In [ ]:
print(t_start, t_end, t_max)

In [ ]:
plt.plot(x2, EFP(x2, *popt))
plt.axhline(0+sigma,linestyle='--', color='r', label = "1$\sigma$")
plt.axhline(0, linestyle = '--', color='g', label = '0')
#plt.scatter(time_new[flags2], counts_new[flags2])
plt.axvline(t_start, ls = '--', color = 'k')
plt.axvline(t_end , ls = '--', color = 'k')
plt.axvline(t_max, ls = '--', color = 'k')
plt.scatter(time_new[flags2], counts_new[flags2], alpha=0.6)
plt.legend()


In [ ]:
t_arr

In [ ]:
mean, sigma

In [ ]:
counts_new[flags]

lc $\rightarrow$ time, rates $\rightarrow$ background correct $\rightarrow$ rebin $\rightarrow$ n-sigma $\rightarrow$ fit $\rightarrow$ t_arr

In [ ]:
def load_lc(filename):
    '''filename : path to the .lc file
    Returns : time, rates'''
    lc = fits.open(filename)
    rates = lc[1].data['RATE']
    time = lc[1].data['TIME']
    return time, rates

In [ ]:
'''time, rates = load_lc("../xsm/data/2021/11/11/calibrated/ch2_xsm_20211111_v1_level2.lc")
rates = background_corrected(time, rates)
n = 3
t_bin_new = 5.0
time_new, rates_new = rebin_lc(time, rates, 1.0, t_bin_new)
flags, mean, sigma = n_sigma(time_new, rates_new, n)
t2, fit2, time_burst, rates_burst = fit_efp(flags, time_new, rates_new, t_bin_new)
'''

In [ ]:
from scipy.interpolate import CubicSpline

In [ ]:
def fit_efp(flags, time, rates, t_bin_new, A0, B0, C0, D0):
    '''Returns fit'''
    #find the largest contiguous interval in flags
    orb_idx, len_arr = orbit_start_indices(np.array(time[flags]).flatten(), t_bin_new)
    burst_start_idx = int(np.argmax(len_arr))
    flags=np.array(flags).flatten()
    burst_flags = flags[orb_idx[burst_start_idx] : orb_idx[burst_start_idx+1]]
    #time_burst = time[burst_flags]
    #rates_burst = rates[burst_flags]
    time_burst = time[flags]
    rates_burst = rates[flags]
    t2 = np.linspace(time_burst[0], time_burst[-1], len(time_burst)*100)
    A0 *= np.sqrt(max(rates_burst))
    B0 *= time_burst[np.argmax(rates_burst)]
    C0 *= np.sqrt(max(rates_burst))
    popt, pcov = curve_fit(EFP, time_burst, rates_burst, p0 = [A0, B0 \
                                                               , C0, D0])
    fit2 = EFP(t2, *popt)
    return t2, fit2, time_burst, rates_burst
#-------------------------------------------
time, rates = load_lc("../xsm/data/2021/11/09/calibrated/ch2_xsm_20211109_v1_level2.lc")
rates = background_corrected(time, rates)
n = 3
t_bin_new = 30.0
time_new, rates_new = rebin_lc(time, rates, 1.0, t_bin_new)
plt.figure(0)
plt.plot(time_new, rates_new)

for i in range(10):
    flags, mean, sigma = n_sigma(time_new, rates_new, n)
    plt.axhline(mean+n*sigma,linestyle='--', color='r')
    plt.axhline(mean, linestyle = '--', color='g')
    t2, fit2, time_burst, rates_burst = fit_efp(flags, time_new, rates_new, t_bin_new, 1*i/10, 1, 1, 0.01)
    plt.figure(1)
    plt.plot(t2, fit2)
    #plt.xlim(1.529e8+72000, 1.529e8+76000)
    #plt.xlim(1.53e8+50000, 1.53e8+65000)
    plt.scatter(time_burst, rates_burst)
    plt.figure(2)
    cs = CubicSpline(time_burst, rates_burst)
    plt.scatter(time_burst, rates_burst)
    plt.plot(time_burst, cs(time_burst))
    plt.show()

look at binning function 

In [ ]:
class lc:
    def __init__(self, filename, t_bin_new):
        self.filename = filename
        self.time, self.lc_rates = load_lc(filename)
        self.bc_rates = background_corrected(self.time, self.lc_rates, 'constant')
        self.binned_time, self.binned_rates = rebin_lc(self.time, self.lc_rates, 1.0, t_bin_new)
        

In [ ]:
lc1 = lc("../xsm/data/2021/11/09/calibrated/ch2_xsm_20211109_v1_level2.lc", 20)

In [ ]:
plt.scatter(lc1.time, lc1.lc_rates)

In [ ]:
def plot_peak_countrates_hist(folder_path):
    filenames = glob.glob(folder_path + '*.lc')
    peak_arr = []
    for lc_file in filenames:
        time, rates = load_lc(lc_file)
        peak_arr.append(max(rates))
    plt.figure(0)
    plt.hist(peak_arr)
    plt.show()
    plt.savefig("peak_hist.png")
    return 0
plot_peak_countrates_hist('/media/pranav/page/Laptop data/Coursework/Semester 8/InterIIT/Extracted lightcurve/')